# Section 1: Data perparation and EDA 

In [ ]:
# Import basic libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
df_train_label=pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
df_train_label.head()

In [ ]:
A=np.array([[1,2],[3,4]])
A

In [ ]:
df_train_label.shape

#### Importing the path to the files

In [ ]:
# path of the files
paths_files = glob("../input/g2net-gravitational-wave-detection/train/*/*/*/*")

In [ ]:
len(paths_files)

Now we look into a particular .npy file as shown below.

In [ ]:
# Loading the first .npy data
data=np.load(paths_files[0])
data

In [ ]:
data.shape

From the above observation we can conclude the following,
1. The sampling rate is 2048 Hz, which means that for each second 2048 samples are given. This fact is already given in the dataset description.
2. Three rows in data variable refer to the 3 sites mentioned in the description of data, and they are: LIGO Hanford (SITE1), LIGO Livingston (SITE2), Virgo (SITE3).
3. In the data variable there are  4096=2086×2  columns. It refers to the total samples generated in the span of 2 seconds.

In [ ]:
print(np.min(data),np.max(data))
# Looking is there is any missing value
df_train_label.isnull().sum()

In [ ]:
df_train_label['target'].hist()

In [ ]:
df_train_label['target'].value_counts()

Motivated by the compact dataset representation in the kaggle notebook given here we also build similar compact dataframe as shown below.

In [ ]:
ids=[]
for filext in paths_files:
    ids.append(filext[filext.rindex('/')+1:\
                              len(filext)].replace('.npy',''))
    
# data frame containing paths and ids of .npy files 
path_df = pd.DataFrame({"id":ids,"path":paths_files})
path_df.head()

In [ ]:
path_df.shape

We know do a join both of the dataframes into a resulting dataframe df

In [ ]:
df=pd.merge(path_df,df_train_label,on='id')
del path_df, df_train_label;
df.head()

In [ ]:
df.shape

#### Visualizing a particular .npy file where target=0 and target=1.

In [ ]:
data1=np.load(df[df['target']==1]['path'].iloc[0])
data0=np.load(df[df['target']==0]['path'].iloc[0])

In [ ]:
fig, axes = plt.subplots(3, 2, sharex=True, figsize=(14,12))
fig.suptitle('Distribution plots')
for i in range(0,data1.shape[0]):
    sns.histplot(ax=axes[i, 0], data=data1[i,:])
    axes[i,0].set_title('SITE'+str(i+1)+'(target=1)')
    sns.histplot(ax=axes[i, 1], data=data0[i,:])
    axes[i,1].set_title('SITE'+str(i+1)+'(target=0)')

It appears that target=1 at SITE1 has higher spread, and target=0 has higher spread at SITE3

#### Going to perform spectrogram analysis

In [ ]:
from scipy import signal

In [ ]:
fig, axes = plt.subplots(3, 2, sharex=True, figsize=(14,12))
fig.suptitle('Distribution plots')
for i in range(0,data1.shape[0]):
    f, t, Sxx = signal.spectrogram(data1[i,:], 2048)
    axes[i, 0].pcolormesh(t, f, Sxx, shading='gouraud')
    axes[i,0].set_ylim([0,30])
    #sns.histplot(ax=axes[i, 0], data=data1[i,:])
    axes[i,0].set_title('SITE'+str(i+1)+'(target=1)')
    f0, t0, Sxx0 = signal.spectrogram(data0[i,:], 2048)
    axes[i, 1].pcolormesh(t0, f0, Sxx0, shading='gouraud')
    axes[i,1].set_title('SITE'+str(i+1)+'(target=0)')
    axes[i,1].set_ylim([0,30])

From the above graphs it is a bit clear that spctrogram of the signals with GW differ siginificantly from signals without GWs.  

In [ ]:
np.max(f)

In [ ]:
print(np.min(t),np.min(t))

Creating Training and validation set

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_val= train_test_split(df, test_size=0.2, random_state=0)

### Now we are going to build input pipeline and the CNN models for the training

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

Directly training our model on the .npy files takes a lot of time because loading the data takes a lot of time than performing the ML computations. Mainly the ML computation are done on a GPU, and loading the data task is done by CPU. Former data pipelines made the GPU wait for the CPU to load the data, leading to performance issues ref.(2). Therefore, the tf.data API enables you to build complex input pipelines from simple, reusable pieces ref.(3). But this (tf.data) API lack the feature of reading .npy files which doesn't fit in the memory. However, I found a solution in the stackover flow, and link of the solution is given here.

It is actually possible to read directly NPY files with TensorFlow instead of TFRecords. The key pieces are tf.data.FixedLengthRecordDataset and tf.io.decode_raw, along with a look at the documentation of the .npy format. For simplicity, let's suppose that a float32 .npy file containing an array with shape (N, K) is given, and you know the number of features K beforehand, as well as the fact that it is a float32 array. An .npy file is just a binary file with a small header and followed by the raw array data (object arrays are different, but we're considering numbers now). In short, you can find the size of this header with a function like this:

In [ ]:
def npy_header_offset(npy_path):
    with open(str(npy_path), 'rb') as f:
        if f.read(6) != b'\x93NUMPY':
            raise ValueError('Invalid NPY file.')
        version_major, version_minor = f.read(2)
        if version_major == 1:
            header_len_size = 2
        elif version_major == 2:
            header_len_size = 4
        else:
            raise ValueError('Unknown NPY file version {}.{}.'.format(version_major, version_minor))
        header_len = sum(b << (8 * i) for i, b in enumerate(f.read(header_len_size)))
        header = f.read(header_len)
        if not header.endswith(b'\n'):
            raise ValueError('Invalid NPY file.')
        return f.tell()

In [ ]:
header_size = npy_header_offset(df['path'].iloc[0])
header_size

In [ ]:
tf_data_train=tf.data.FixedLengthRecordDataset( df_train['path'], 3*4096*tf.float64.size,\
                                         header_bytes=header_size, num_parallel_reads=64)
tf_data_val=tf.data.FixedLengthRecordDataset( df_val['path'], 3*4096*tf.float64.size,\
                                         header_bytes=header_size, num_parallel_reads=64)

In [ ]:
tf_data_train = tf_data_train.map(lambda s: tf.reshape(\
                                                       tf.io.decode_raw(s, tf.float64),\
                                                       (3,4096)))
tf_data_val = tf_data_val.map(lambda s: tf.reshape(\
                                                       tf.io.decode_raw(s, tf.float64),\
                                                       (3,4096)))
tf_data_train

In [ ]:
!python -m pip install gwpy
!pip install astropy==4.2.1

In [ ]:
from gwpy.timeseries import TimeSeries
data=np.load(df_train['path'].iloc[0])
d1 = TimeSeries(data[0,:], sample_rate=2048)
white_data = d1.whiten(window=("tukey",0.2)) # whiten-function has a built-in window fu
plt.scatter(np.arange(0,4096),white_data)
np.array(white_data)

In [ ]:
def whitening_fn(s):
    #print(s)
    hann=tf.signal.hann_window(4096, periodic=True, dtype=tf.dtypes.float64)
    spec1=tf.signal.rfft(s[0,:]*hann)
    spec2=tf.signal.rfft(s[1,:]*hann)
    spec3=tf.signal.rfft(s[2,:]*hann)
    mag1 = tf.math.sqrt(tf.math.real(spec1*tf.math.conj(spec1)))
    mag2 = tf.math.sqrt(tf.math.real(spec2*tf.math.conj(spec2)))
    mag3 = tf.math.sqrt(tf.math.real(spec3*tf.math.conj(spec3)))
    x=tf.reshape(tf.math.real(tf.signal.ifft(spec1/tf.cast(mag1,tf.complex128))\
                             )* np.sqrt(4096/2),shape=(1,2049))
    y=tf.reshape(tf.math.real(tf.signal.ifft(spec2/tf.cast(mag2,tf.complex128))\
                             )* np.sqrt(4096/2),shape=(1,2049))
    z=tf.reshape(tf.math.real(tf.signal.ifft(spec3/tf.cast(mag3,tf.complex128))\
                             )* np.sqrt(4096/2),shape=(1,2049))
    c = tf.concat([x, y, z], axis=0)
    #print(c)
    return c

In [ ]:
tf_data_train=tf_data_train.map(whitening_fn)
tf_data_val=tf_data_val.map(whitening_fn)

In [ ]:
for i in tf_data_train.take(1):
    plt.scatter(np.arange(0,2049),i[0,:].numpy())
    plt.show()
    plt.scatter(np.arange(0,2049),i[1,:].numpy())
    plt.show()
    plt.scatter(np.arange(0,2049),i[2,:].numpy())
    plt.show()

In [ ]:
#def tf_function(s):
#    x=tf.numpy_function(whitening_fn, [s], tf.float64)
#    return x

#def whitening_fn(s):
#    h1 = TimeSeries(s[0,:], sample_rate=2048)
#    h1_whiten = h1.whiten(window=("tukey",0.2))
#    h2 = TimeSeries(s[1,:], sample_rate=2048)
#    h2_whiten = h2.whiten(window=("tukey",0.2))
#    h3 = TimeSeries(s[2,:], sample_rate=2048)
#    h3_whiten = h3.whiten(window=("tukey",0.2))
#    s[0,:]=h1_whiten
#    s[1,:]=h2_whiten
#    s[2,:]=h3_whiten
#    return s

#tf_data_train=tf_data_train.map(tf_function)
#tf_data_val=tf_data_val.map(tf_function)

Standardized the training dataset

In [ ]:
def parse_standardize(s):
    # mean and variance calculations row wise
    mean,variance=tf.nn.moments(s, axes=[1],keepdims=True)
    s = (s - mean) / tf.sqrt(variance)
    return s
tf_data_train=tf_data_train.map(parse_standardize)

In [ ]:
tf_data_val=tf_data_val.map(parse_standardize)

Here we are showing first normalized entry of the training dataset

In [ ]:
j=0
for i in tf_data_train.take(2):
    if j==1:
        print(i.numpy())
    j=j+1

In [ ]:
data=np.load(df_train['path'].iloc[1])

Checking whether the normalization is done correctly or not

In [ ]:
data[0,:]=(data[0,:]-np.mean(data,axis=1)[0])/np.std(data,axis=1)[0]
data[1,:]=(data[1,:]-np.mean(data,axis=1)[1])/np.std(data,axis=1)[1]
data[2,:]=(data[2,:]-np.mean(data,axis=1)[2])/np.std(data,axis=1)[2]
data

In [ ]:
tf_data_train=tf_data_train.map(lambda s: tf.reshape(s,shape=(3,2049,1)))

In [ ]:
tf_data_val=tf_data_val.map(lambda s: tf.reshape(s,shape=(3,2049,1)))

In [ ]:
for i in tf_data_train.take(1):
    print(i.numpy())

#### Write about the axes of **tf.nn.moments**

In [ ]:
#xs = tf.convert_to_tensor(np.array([[[-1,3,2], [-3,1,3]],[[2,-7,4],[5,7, 6]]]), dtype = tf.float32)
#fc_mean, fc_var = tf.nn.moments(xs, axes = [0], keepdims=True)
#print(fc_mean)

In [ ]:
#xs = tf.convert_to_tensor(np.array([[[-1,3,2], [-3,1,3]],[[2,-7,4],[5,7, 6]]]), dtype = tf.float32)
#fc_mean, fc_var = tf.nn.moments(xs, axes = [0,2], keepdims=True)
#print(fc_mean)

Now going to zip the target column with the tensorflow dataset

In [ ]:
tf_data_train= tf.data.Dataset.zip((tf_data_train,\
                             tf.data.Dataset.from_tensor_slices(df_train['target'])))

In [ ]:
i=0
for data, target in tf_data_train.take(3):
    print("tf_data_train")
    print(data.numpy(),target.numpy())
    print("df_train")
    print(np.load(df_train['path'].iloc[i]),df_train['target'].iloc[i])
    i=i+1

In [ ]:
tf_data_val= tf.data.Dataset.zip((tf_data_val,\
                             tf.data.Dataset.from_tensor_slices(df_val['target']))) 
for data, target in tf_data_val.take(3):
    print(data.numpy(),target.numpy())

In [ ]:
train_data = tf_data_train.batch(64).prefetch(tf.data.AUTOTUNE)
train_data

In [ ]:
val_data = tf_data_val.batch(32).prefetch(tf.data.AUTOTUNE)
val_data

In [ ]:
model = Sequential()
model.add(Conv2D(16,input_shape=(3, 2049,1), kernel_size=(3,16),strides=(1,1),activation=None))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,4),strides=(1,4)))
model.add(Activation("relu"))
model.add(Conv2D(32,kernel_size=(1,8),strides=(1,1),dilation_rate=4,activation=None))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,4),strides=(1,4)))
model.add(Activation("relu"))
model.add(Conv2D(64,kernel_size=(1,8),strides=(1,1),dilation_rate=4,activation=None))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,4),strides=(1,4)))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer = Adam(lr=1e-3),loss='binary_crossentropy',metrics=['AUC'])
model.summary()

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',factor=np.sqrt(0.01), cooldown=0,\
                               patience=3, min_lr=0.5e-8,mode='min')
# This file path where the best weights will be saved
checkpoint_filepath = './weights10.best.hdf5'
# It saves the weights which shows the highest validation accuracy
modelchck=ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,verbose=1,
                          monitor='val_auc',mode='max',save_best_only=True)

model.fit(train_data, validation_data=val_data, epochs = 10, callbacks=[lr_reducer,modelchck])

In [ ]:
# The model weights (that are considered the best) are loaded into the model.
checkpoint_filepath = '../input/saved-weights/weights10.best.hdf5'
model.load_weights(checkpoint_filepath)

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',factor=np.sqrt(0.1), cooldown=0,\
                               patience=3, min_lr=0.5e-8,mode='min')
# This file path where the best weights will be saved
checkpoint_filepath = './weights20.best.hdf5'
# It saves the weights which shows the highest validation accuracy
modelchck=ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,verbose=1,
                          monitor='val_auc',mode='max',save_best_only=True)

model.fit(train_data, validation_data=val_data, epochs = 10, callbacks=[lr_reducer,modelchck])

In [ ]:
# path of the files
test_files = glob("../input/g2net-gravitational-wave-detection/test/*/*/*/*")
#paths_files

In [ ]:
ids=[]
for filext in test_files:
    ids.append(filext[filext.rindex('/')+1:\
                              len(filext)].replace('.npy',''))
    
# data frame containing paths and ids of .npy files 
test_df = pd.DataFrame({"id":ids,"path":test_files})
test_df.head()

In [ ]:
tf_data_test=tf.data.FixedLengthRecordDataset( test_df['path'], 3*4096*tf.float64.size,\
                                         header_bytes=header_size, num_parallel_reads=4)
tf_data_test = tf_data_test.map(lambda s: tf.reshape(\
                                                       tf.io.decode_raw(s, tf.float64),\
                                                       (3,4096)))
tf_data_test=tf_data_test.map(whitening_fn)
tf_data_test=tf_data_test.map(parse_standardize)
tf_data_test=tf_data_test.map(lambda s: tf.reshape(s,shape=(3,2049,1)))
test_data = tf_data_test.batch(32).prefetch(buffer_size=64)
y_pred=model.predict(test_data)

In [ ]:
y_pred.flatten()

In [ ]:
output = pd.DataFrame({'Id': test_df.id, 'target': y_pred.flatten()})
output.head()

In [ ]:
output.to_csv('./testing_submission.csv', index=False)
print("Your submission was successfully saved!")